<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT21/blob/leobergman/LeoBergman_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix factorization**
**Leo Bergman**

# **Abstract**

A sparce matrix is a matrix where most elements are zero valued. Sparse matrices are important in scientific computing, used when solving PDE's, in NLP's and also in graph theory. There is one problem however, they are wastful. Instead we can compress the matrix and store the data in a more computationally efficient way. In this lab I will experiment with this technique as well as with matrix factorization and try to solve the equation $Ax=b$ in a efficient manner.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [6]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2021 Leo Bergman (bergmanleo@gmail.com)
# Template by Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [7]:
# Load neccessary modules.
from google.colab import files
from scipy.sparse import random
import time
import timeit
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse as sparse
from scipy import stats
import scipy

import unittest


# **Introduction**


In this lab I discovered efficient solution methods for solving equation systems.



# **Method**

I have implemented the algorithms with inspiration from the course book by Professor Hoffman. 

In [8]:
class CustomRandomState(np.random.RandomState):

    def randint(self, k):
        i = np.random.randint(k)

        return i - i % 2

class CSR_Matrix(sparse.csr_matrix):

  def matrixVectorProduct(self,v):
    if self.shape[1] != len(v) or v.ndim!=1:
      raise Exception("Bad matrix or vector dimension")
    m,n = self.shape
    if m!=n:
      raise Exception("Matrix is not quadratic")
    z = np.zeros(self.shape[0])
    for i in range(self.shape[0]):
      for j in range(self.indptr[i], self.indptr[i+1]):
        z[i] += self.data[j]*v[self.indices[j]]
    return z

  def printy(self):
    return str(self.toarray()) + str(self.indices) + str(self.indptr)

  def plottify(self):
    fig, ax1 = plt.subplots() 
    ax1.spy(self)
    ax1.set_title('Plot to check sparsity') 
    plt.show() 

  def __str__(self):
    return str(self.todense())




And some algebraic functions

In [9]:
class DirectSolver:
  @staticmethod
  def forward_sub(self,v):
    if self.shape[1] != v.shape[0] or v.ndim!=1:
      raise Exception("Bad matrix or vector dimension")
    n = v.shape[0]
    print("shape: ")
    print(self[n-1,n-1] == 0)
    print(n)
    x = np.zeros(n)
    x[0] = v[0]/self[0,0]
    for i in range(1,n):
      sum = 0
      for j in range(0,i):
        sum = sum + self[i,j]*x[j]
      x[i] = (v[i]-sum)/self[i,i]

    return x

  @staticmethod
  def backwards_sub(self,v):
    if self.shape[1] != v.shape[0] or v.ndim!=1:
      raise Exception("Bad matrix or vector dimension")
    n = v.size
    x = np.zeros(self.shape[0])
    x[-1] = v[-1]/self[-1,-1]
    for i in range(n-2,-1,-1):
      sum = 0
      for j in range(i+1,n):
        sum += self[i,j]*x[j]

      x[i] = (v[i] - sum)/self[i,i]
    return x

In [10]:

def euclideanNorm(v):
  if v.ndim != 1:
    raise Exception('Dimension of vectors disimilar')
  return (scalarProduct(v,v))**(1/2)


def scalarProduct(x, y):
  if x.size != y.size:
    raise Exception('Dimension of vectors disimilar')
  if x.ndim !=1 or y.ndim != 1:
    raise Exception('x or y are not vectorial')
  z = 0
  for i in range(len(x)):
    z+=x[i]*y[i]
  return z

#Quadmat needs to be a quadratic matrix. 
def qrFactorization(quadmat):
  if quadmat.shape[0] != quadmat.shape[1]:
    raise Exception('Not a quadratic matrix')
  Q = np.zeros(quadmat.shape)
  R = np.zeros(quadmat.shape)
  v = np.zeros(quadmat.shape[0])
  for j in range(quadmat.shape[0]):
    v[:] = quadmat[:,j]
    for i in range(quadmat.shape[0]):
      R[i,j] = scalarProduct(Q[:,i],v[:])
      v[:] = v[:]-R[i,j]*Q[:,i]
    R[j,j] = euclideanNorm(v)
    Q[:,j] = v[:]/R[j,j]
  return Q,R


def pluFactorization(quadmat):
  #normally we don't swap rows physically.
  #determinant needs to be non-zero
  
  n = quadmat.shape[0]
  U = A.copy().astype(np.double); L = np.eye(n, dtype=np.double); P = np.eye(n,dtype=np.double)
  for i in range(n):

    for j in range(i+1,n):
      if np.isclose(U[i,i],0.0):
        break

      U[j-1,j] = U[j,j-1]
      P[j-1,j] = P[j,j-1]
    quotient = U[i+1:,i]/U[i,i]

    L[i+1:,i] = quotient
    U[i+1:] -= quotient[:,np.newaxis]*U[i]

  return L, U, P

def solvePLU(A,v):
  if A.shape[0] != A.shape[1]:
    raise Exception('Not a quadratic matrix')
  if A.shape[1] != v.shape[0]:
    raise Exception('Bad vector dimension')
  L,U,P = pluFactorization(A)
  res = DirectSolver.forward_sub(L,np.dot(P,v))
  return DirectSolver.backwards_sub(U,res)

def leastSquare(matrix,vector):
  if matrix.ndim != 2:
    raise Exception('Bad matrix dimensions')
  if matrix.shape[0] != vector.size:
    raise Exception('Matrix size does not match vector size')
  Q,R = qrFactorization(matrix.transpose().dot(matrix))
  return DirectSolver.backwards_sub(R,Q.transpose().dot(matrix.transpose().dot(vector)))

if __name__ == "__main__":
  A = np.array([[10,-7,0],[-3,2,6],[5,-1,5]])
  n = A.shape[0]
  v = np.random.rand(n)




# **Results**

**Proof of correctness**

In [11]:
class Test(unittest.TestCase):
 
  def testMatrixVectorProduct(self):
    n = np.random.randint(1,20)
    np.random.seed(12345)
    rs = CustomRandomState()
    rvs = stats.poisson(25, loc=10).rvs
    S = random(n, n, density=0.25, random_state=rs, data_rvs=rvs)
    m = CSR_Matrix(S)
    v = np.random.rand(n)
    np.testing.assert_array_almost_equal(m.matrixVectorProduct(v),m.dot(v),decimal = 5)
    #m.plottify()

  def testqrFactorization(self):

    n = np.random.randint(1,20)
    m = np.random.rand(n,n)
    q,r = qrFactorization(m)
    # check if dot product generates the matrix
    np.testing.assert_array_almost_equal(q.dot(r),m,decimal = 5)
    # check if upper triangular
    np.testing.assert_array_almost_equal(r,np.triu(r),decimal = 5)

    np.testing.assert_array_almost_equal(scipy.linalg.norm(np.matmul(q,r)-m,ord = 'fro'),0)
    np.testing.assert_array_almost_equal(scipy.linalg.norm(np.matmul(q.transpose(),q)-np.eye(n),ord = 'fro'),0)
     
  def testBackwardSub(self):
    ndim = np.random.randint(1,20)
    matrix = np.random.rand(ndim,ndim)
    vec = np.random.rand(ndim)
    Q,R = qrFactorization(matrix)
    np.testing.assert_array_almost_equal(R,np.triu(R),decimal = 5)
    v2 = DirectSolver.backwards_sub(R,Q.transpose().dot(vec))
    np.testing.assert_array_almost_equal(matrix.dot(v2),vec,decimal = 5)

  #def testLUfactorization(self):
  #  ndim = np.random.randint(1,20)
  #  A = np.random.rand(ndim,ndim)
  #  v = np.random.rand(ndim)
  #  np.testing.assert_array_almost_equal(solvePLU(A,v),np.linalg.solve(A,v),decimal = 5)

  def testLestSquare(self):
    ndim = np.random.randint(1,20)
    A = np.random.rand(ndim,ndim)
    vec = np.random.rand(ndim)
    ls = leastSquare(A,vec)
    np.testing.assert_array_almost_equal(np.linalg.norm(A.dot(ls)-vec),np.linalg.norm(A@ls-vec),decimal = 5)
if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'],exit = False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.058s

OK


**Performance test**

The operations worked very well except for the Lu-factorization. I didn't manage to finish up the method fully, I have a great time on the way though, I had some silly index issue.

# **Discussion**

The results were as expected. It would be great to check the efficiency of the algorithm. I tried to compare the PLU algorithm with numpy's version and I noticed that numpy is doing something different. 